<a href="https://colab.research.google.com/github/anachavesv8/Bootcamp_Blue/blob/main/Limpeza_NPL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import random
import math

import matplotlib.pyplot as plt
import seaborn as sns


import numpy as np
import pandas as pd
import re

import sklearn

from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split

from sklearn.metrics.pairwise import cosine_similarity

from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [3]:
train = pd.read_csv('/content/drive/MyDrive/Datasets/train.tsv', sep = '\t')
test = pd.read_csv('/content/drive/MyDrive/Datasets/test.tsv', sep = '\t')

def data(n, seed):

    datas = []
    
    random.seed(seed)
    for i in range(n):
        dia_maximo = 30
        dia_minimo = 1
        mes_maximo = 12
        mes_minimo = 1
        
        dia = int(round(random.random() * (dia_maximo - dia_minimo) + dia_minimo, 0))
        mes = int(round(random.random() * (mes_maximo - mes_minimo) + mes_minimo, 0))
        
        datas.append(str(dia)+'-'+str(mes)+'-2018')
        
    return datas

def estoque(n, seed):

    np.random.seed(seed)
    mu, sigma = 1, 20
    s = np.random.normal(mu, sigma, n)
    s[s < 0] = s[s < 0] * -0.5
    s = s.astype(int)
    s[s < 1] = 1
    
    return s

train['date']  = data(n = train.shape[0], seed = 10)
train['stock'] = estoque(n = train.shape[0], seed = 10)

test['date']  = data(n = test.shape[0], seed = 15)
test['stock'] = estoque(n = test.shape[0], seed = 15)

train.drop('train_id', axis=1, inplace=True)
train['item_condition_id'] = train['item_condition_id'].astype('str')

### Objetivo: Entender o comportamento dos dados através de visualizações

Podemos dizer que a variável 'price' é a variável target, ou seja, a variável que devemos prever. Dessa forma, a maior parte desta análise exploratória envolverá esta variável.

In [4]:
from numpy.lib.function_base import delete
rotulos = [i for i in train['category_name'].fillna('missing/missing/missing').str.split("/")]
rotulos

gen_cat = []
sub1_cat = []
sub2_cat = []

for i in range(0,len(rotulos)):
  gen_cat.append(rotulos[i][0])
  sub1_cat.append(rotulos[i][1])
  sub2_cat.append(rotulos[i][2])

train['gen_cat'] = gen_cat
train['sub1_cat'] = sub1_cat
train['sub2_cat'] = sub2_cat

del gen_cat
del sub1_cat
del sub2_cat

In [5]:
train

,name,item_condition_id,category_name,brand_name,price,shipping,item_description,date,stock,gen_cat,sub1_cat,sub2_cat
0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet,18-6-2018,27,Men,Tops,T-shirts
1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...,18-3-2018,15,Electronics,Computers & Tablets,Components & Parts
2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...,25-10-2018,14,Women,Tops & Blouses,Blouse
3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...,20-3-2018,1,Home,Home Décor,Home Décor Accents
4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity,16-5-2018,13,Women,Jewelry,Necklaces
...,...,...,...,...,...,...,...,...,...,...,...,...
1482530,Free People Inspired Dress,2,Women/Dresses/Mid-Calf,Free People,20.0,1,"Lace, says size small but fits medium perfectl...",13-10-2018,2,Women,Dresses,Mid-Calf
1482531,Little mermaid handmade dress,2,Kids/Girls 2T-5T/Dresses,Disney,14.0,0,Little mermaid handmade dress never worn size 2t,6-10-2018,10,Kids,Girls 2T-5T,Dresses
1482532,21 day fix containers and eating plan,2,Sports & Outdoors/Exercise/Fitness accessories,NaN,12.0,0,"Used once or twice, still in great shape.",6-8-2018,15,Sports & Outdoors,Exercise,Fitness accessories
1482533,World markets lanterns,3,Home/Home Décor/Home Décor Accents,NaN,45.0,1,There is 2 of each one that you see! So 2 red ...,13-2-2018,20,Home,Home Décor,Home Décor Accents


In [6]:
#Instalando a biblioteca
!pip install spacy

import spacy
import spacy.cli

spacy.cli.download("en_core_web_sm")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1482535 entries, 0 to 1482534
Data columns (total 12 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   name               1482535 non-null  object 
 1   item_condition_id  1482535 non-null  object 
 2   category_name      1476208 non-null  object 
 3   brand_name         849853 non-null   object 
 4   price              1482535 non-null  float64
 5   shipping           1482535 non-null  int64  
 6   item_description   1482531 non-null  object 
 7   date               1482535 non-null  object 
 8   stock              1482535 non-null  int64  
 9   gen_cat            1482535 non-null  object 
 10  sub1_cat           1482535 non-null  object 
 11  sub2_cat           1482535 non-null  object 
dtypes: float64(1), int64(2), object(9)
memory usage: 135.7+ MB


In [8]:
train1 = train.copy() 
train1.isna().sum().loc[train1.isna().sum()>0].sort_values()

item_description         4
category_name         6327
brand_name          632682
dtype: int64

In [11]:
train1.dropna(subset=['item_description', 'category_name'], inplace=True)

In [12]:
train1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1476204 entries, 0 to 1482534
Data columns (total 12 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   name               1476204 non-null  object 
 1   item_condition_id  1476204 non-null  object 
 2   category_name      1476204 non-null  object 
 3   brand_name         846982 non-null   object 
 4   price              1476204 non-null  float64
 5   shipping           1476204 non-null  int64  
 6   item_description   1476204 non-null  object 
 7   date               1476204 non-null  object 
 8   stock              1476204 non-null  int64  
 9   gen_cat            1476204 non-null  object 
 10  sub1_cat           1476204 non-null  object 
 11  sub2_cat           1476204 non-null  object 
dtypes: float64(1), int64(2), object(9)
memory usage: 146.4+ MB


In [13]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [14]:
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
vectorizer = CountVectorizer(analyzer='word', ngram_range=(1, 1))
tfidf_transformer = TfidfTransformer()

In [17]:
titulos = train1.name
descricao  = train1.item_description.tolist()

In [19]:
# funcao para tratar cada palavra e remover stopwords
def processamento(tokens):
    
    token_processado = []
    for token in tokens:
        token = token.lower()
        token = lemmatizer.lemmatize(token)
        
        if token not in stop_words:
            token = stemmer.stem(token)
            token_processado.append(token)
        
    return token_processado

documentos = []
for titulos in titulos:
    
    # expressao regular para remover pontuacoes do texto
    descricao = re.sub(r'[^\w\s]','', titulos)
    tokens = processamento(word_tokenize(titulos))
    
    documentos.append(' '.join(tokens))

In [20]:
X_train_counts = vectorizer.fit_transform(documentos)
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

X = X_train_tfidf

In [21]:
documentos

['mlb cincinnati red shirt size xl',
 'razer blackwidow chroma keyboard',
 'ava-viv blous',
 'leather hors statu',
 '24k gold plate rose',
 'bundl item request ruie',
 'acacia pacif tide santorini top',
 'girl cheer tumbl bundl 7',
 'girl nike pro short',
 'porcelain clown doll checker pant vtg',
 'smashbox primer',
 'new v pi k bodi mist',
 'black skater dress',
 'sharpen eras',
 'hold dogs2016 minnetonka boot',
 'sephora tart birthday gift',
 'glitter eyeshadow',
 "new : babi k'tan activ babi carrier",
 "face limit `` merri macaroon ''",
 'cream/ beig front cross shirt',
 'torrid nautic peplum tube top',
 'nwt v ultim sport bra 34ddd',
 'galaxi s7 edg ( unlock ) 32gb',
 'tripl car charger',
 'black red basebal tee',
 'air jordan carmin 6',
 'otterbox defend iphon 6 plus/6 plu',
 'lularo black white polka dot',
 'forever21 floral romper strapless',
 'kendra bundl',
 'face better sex mascara qty3',
 'new lularo leg , star',
 'pink bra 36d',
 'caudalí beauti elixir spray',
 'four puppi 

In [22]:
Count_titulos = vectorizer.fit_transform((train1['name']))
Count_titulos

<1476204x105308 sparse matrix of type '<class 'numpy.int64'>'
	with 6210528 stored elements in Compressed Sparse Row format>